In [17]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from datetime import datetime
import re
import random
import math

In [18]:
# loading data
codes = pd.read_csv('../results/patients_w_codes.csv')

In [19]:
# version with dates into strings
codes2 = pd.read_csv('../results/patients_w_codes2.csv')

In [20]:
# to use a dict with the dates in the strings
def separate_date(df):
    df['first_event_date'] = 'NA'
    for i in range(len(df)):
        x = re.split(']', df['code'][i].replace('[', ''))
        df['first_event_date'][i] = x[0]
        df['code'][i] = x[1]
    return df

In [21]:
separate_date(codes2)
codes2

,id,code,first_event_date
0,005cad4958846409,B42A40A2H1F10A15B21B7B12I24I7F6B28B128A4D77B64...,2020-08-14
1,0065b57e5a9e784f,A37B234F4B4B8B2F2B2F3B4F0G3B2I8G17I215I82B9B23A,2020-03-26
2,006e5a1bf72e8b73,B56B7B35B11B3B11B10B13B14B8A178I21I123B6B5B6B5...,2020-06-02
3,009d11a2497f1d47,A334C142J,2020-10-14
4,00a5cf0620b6adfc,A201J213A,2020-10-26
...,...,...,...
3298,ff32e199c9482c20,B196B12B21B6F1B7B1I6B6A15B7B7I70B14B78B7B7B8B5...,2020-05-07
3299,ff46357762ea097e,B92B99B12B9B6B4B32F6I4B18I232C4A14B17I79D36B3B...,2020-04-20
3300,ff9318bd975b7050,A16A64B74B14A197I,2020-08-10
3301,ffabda76a8fbaecd,B24B182A12B13B7B6F1B0I7B1B5B7F1I56B15F218B4F10...,2020-05-05


In [22]:
# Function to split strings
def split_string(s):
    """
    s is a string code. Returns two lists, with letters (A) and numbers (B).
    """
    S = []
    for x in re.split('(\d+)', s):
        try:
            S.append(int(x))
        except ValueError:
            S.append(x)
    A, B = S[::2], S[1::2]
    return A, B

In [75]:
def case1(A, B, pos):
    """
    Recognises case 1 reinfections (+|-|+).
    Added some lines to detect certain cases 2.
    """
    if A[0] in pos:
        i = 0
        c = B[i]
        # to find case 2 before case 1
        #if A[i] in pos and A[i+1] in pos and B[i] >= 90:
        #    return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        if A[i+1] in pos and c >= 90:
            return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        while A[i+1] != 'B':
            i += 1
            c += B[i]
            # to find case 2 before case 1
            if A[i+1] in pos and c >= 90:
                return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        # here the first B is found
        i += 1
        c += B[i]
        while A[i+1] not in pos:
            i += 1
            c += B[i]
        if c >= 90:
            return 'Case 1', A[i+1:len(A)], B[i+1:len(B)]
        elif c >= 60:
            return 'Possible case 1', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return '', A[i+1:len(A)], B[i+1:len(B)]
    else:
        i = 0
        while A[i] not in pos:
            i += 1
        c = B[i]
        # same as above
        if A[i+1] in pos and c >= 90:
            return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        while A[i+1] != 'B':
            i += 1
            c += B[i]
            if A[i+1] in pos and c >= 90:
                return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
        i += 1
        c += B[i]
        while A[i+1] not in pos:
            i += 1
            c += B[i]
        if c >= 90:
            return 'Case 1', A[i+1:len(A)], B[i+1:len(B)]
        elif c >= 60:
            return 'Possible case 1', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return '', A[i+1:len(A)], B[i+1:len(B)]

In [193]:
def case2(A, B, pos):
    """
    Recognises case 2 reinfections (+|+)
    """
    c = 0
    if A[0] in pos:
        for i in range(len(B)):
            if A[i+1] not in pos:
                if bool([x for x in A[i+1:len(A)] if(x in pos)]) == False:
                    if c >= 60:
                        return 'Possible Case 2', A[i:len(A)], B[i:len(B)]
                    else:
                        return '', A[i:len(A)], B[i:len(B)]
                c += B[i]
            else:
                if c + B[i] >= 90:
                    return 'Case 2', A[i+1:len(A)], B[i+1:len(B)]
                c = B[i]
        if c >= 60:
            return 'Possible Case 2', A[i+1:len(A)], B[i+1:len(B)]
        else:
            return '', A[i+1:len(A)], B[i+1:len(B)]
    else:
        i = 0
        while A[i] not in pos:
            i += 1
        for j in range(i, len(B)):
            if A[j+1] not in pos:
                if bool([x for x in A[j+1:len(A)] if(x in pos)]) == False:
                    if c >= 60:
                        return 'Possible Case 2', A[j:len(A)], B[j:len(B)]
                    else:
                        return '', A[j:len(A)], B[j:len(B)]
                c += B[j]
            else:
                if c + B[j] >= 90:
                    return 'Case 2', A[j+1:len(A)], B[j+1:len(B)]
                c = B[j]
        if c >= 60:
            return 'Possible Case 2', A[j+1:len(A)], B[j+1:len(B)]
        else:
            return '', A[j+1:len(A)], B[j+1:len(B)]

In [203]:
# adding multiple reinfection management
def classif(s, pos = ['A', 'C', 'D']):
    """
    Takes a string and classify it into reinfection cases 1 or 2
    """
    A, B = split_string(s)
    L = []
    while len(A) > 1:
        # indices stores a list between the first and last positive in the string
        indices = [i for i, x in enumerate(A) if x in pos]
        # b will be used to break the loop if there are no positives in the remaining string
        b = [x for x in A[1:len(A)] if(x in pos)]
        if 'B' in A[min(indices):max(indices)]:
            r, A, B = case1(A, B, pos)
            if r != '':
                L.append(r)
            print('Case 1', A)
        elif bool(b) == False:
            break
        else:
            r, A, B = case2(A, B, pos)
            print('Case 2', A)
            if r != '':
                L.append(r)
    return L

In [240]:
# trying random strings
r = random.randint(0,3303)
print(codes.iloc[r]['id'], '\n')
s = codes.iloc[r]['code']

A, B = split_string(s)

print(s, '\n')

classif(s)

e5edb3f6c4fa8f06 

B188B6F4B6B9A5A51I21I168B12B5F4B8B6B7B21I104F7F13F2B6B4A 

Case 1 ['A']


['Case 1']

In [199]:
# applying function
d = {}
for x in codes['id']:
    d[str(x)] = classif(codes.loc[codes['id'] == x]['code'].values[0])

In [200]:
import csv

with open('../results/dict.csv', 'w') as f:
    for key in d.keys():
        f.write("%s, %s\n" % (key, d[key]))

In [201]:
N = list(d.values())
N = [str(x) for x in N]
freq = {}
for items in N:
    freq[items] = N.count(items)
freq

{"['Case 1']": 2109,
 "['Case 2']": 996,
 "['Case 2', 'Case 2']": 5,
 "['Case 1', 'Case 1']": 99,
 "['Possible case 1']": 26,
 "['Case 2', 'Case 1']": 17,
 '[]': 18,
 "['Case 1', 'Possible case 1']": 5,
 "['Possible case 1', 'Case 1']": 10,
 "['Case 1', 'Case 2']": 2,
 "['Possible Case 2']": 8,
 "['Case 1', 'Case 1', 'Possible case 1']": 2,
 "['Case 1', 'Possible case 1', 'Case 1']": 2,
 "['Case 2', 'Possible Case 2']": 1,
 "['Case 1', 'Case 1', 'Case 1']": 1,
 "['Possible case 1', 'Possible case 1']": 2}

In [202]:
d

{'005cad4958846409': ['Case 1'],
 '0065b57e5a9e784f': ['Case 1'],
 '006e5a1bf72e8b73': ['Case 1'],
 '009d11a2497f1d47': ['Case 2'],
 '00a5cf0620b6adfc': ['Case 2'],
 '00cad70e300b78c8': ['Case 2'],
 '00f31c11a3a6f08d': ['Case 2'],
 '011e826ab054591f': ['Case 2'],
 '012a57320c31a43f': ['Case 1'],
 '0134889c8d47909e': ['Case 2'],
 '01394866d1e64c72': ['Case 1'],
 '0139873673801ea0': ['Case 1'],
 '01422e9f5d9ad070': ['Case 2'],
 '015271b745049623': ['Case 1'],
 '015d56e832d129b0': ['Case 2'],
 '0165639a7fb80826': ['Case 1'],
 '0180c7c2db07677c': ['Case 1'],
 '0197651a06a237d7': ['Case 2'],
 '0198a2202a0f910a': ['Case 1'],
 '019d07016a193c91': ['Case 1'],
 '01a300fb5234bab8': ['Case 2'],
 '01b1d27f1734da73': ['Case 1'],
 '01c190b6006937c5': ['Case 2'],
 '01cd65c858e5b632': ['Case 2'],
 '022c2f46c2667c5a': ['Case 2'],
 '022dfea715f9b986': ['Case 1'],
 '02369f6899736814': ['Case 2'],
 '02474e25f210a54e': ['Case 1'],
 '025541d83c991e99': ['Case 1'],
 '027b9184f7dbddbc': ['Case 1'],
 '029a76ff